##Fact ORDERS

In [0]:
from delta.tables import DeltaTable

In [0]:
df = spark.sql("select * from databricksete.silver.orders_silver")
df.display()

In [0]:
df_dimcus = spark.sql("select DimCustomerKey ,customer_id as dim_customer_id from databricksete.gold.DimCustomers ")
df_dimcus.display()

df_dimpro = spark.sql("select  product_id  as DimProductkey ,product_id  as dim_product_id  from databricksete.gold.DimProducts")

In [0]:
df_dimpro.display()

In [0]:
df_fact =  df.join(df_dimcus,df['customer_id']== df_dimcus['dim_customer_id'],how = 'Left').join(df_dimpro,df['product_id']== df_dimpro['dim_product_id'])

In [0]:
df_fact_new  = df_fact.drop('dim_customer_id','dim_customer_id')

In [0]:
from delta.tables import DeltaTable

In [0]:
path = "abfss://gold@salesstaradlsstorageacc.dfs.core.windows.net/factorders"

In [0]:
df_fact_new = df_fact_new.dropDuplicates(["order_id", "DimCustomerKey", "DimproductKey"])


In [0]:
from delta.tables import DeltaTable

if spark.catalog.tableExists("databricksete.gold.fact_orders"):
    dlt_obj = DeltaTable.forName(spark, "databricksete.gold.fact_orders")
    dlt_obj.alias("trg").merge(
        df_fact_new.alias("src"),
        "trg.order_id = src.order_id AND trg.DimCustomerKey = src.DimCustomerKey AND trg.DimproductKey = src.DimproductKey"
    ) \
    .whenMatchedUpdateAll() \
    .whenNotMatchedInsertAll() \
    .execute()
else:
    df_fact_new.write.mode("overwrite").format("delta") \
        .saveAsTable("databricksete.gold.fact_orders")
